In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
np.diag(np.ones(5))

In [ ]:
import pickle

In [ ]:
import xarray as xr

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from matplotlib import cm

In [ ]:
from datetime import datetime

In [ ]:
import numpy as np
from math import ceil
import yaml

In [ ]:
conf = yaml.load(open('./../conf-all.yaml','r'), Loader=yaml.SafeLoader)

In [ ]:
select_sites = conf['dataset_parameters']['select_sites']

In [ ]:
norms = pickle.load(open('./../experiments_2/sacred/12/var_norms.pkl','rb'))

In [ ]:
dict(zip(norms['std_norm']['shift_targets_WATER_VOLUME']['std'].to_dict()['coords']['global_sites']['data'],norms['std_norm']['shift_targets_WATER_VOLUME']['std'].to_dict()['data']))

In [ ]:
ds_data = pickle.load(open('./../data/ds_data.pkl','rb'))
ds_meta = pickle.load(open('./../data/ds_meta.pkl','rb'))

In [ ]:
ds = xr.open_dataset('./../data/data_all.nc').compute()

In [ ]:
ds['targets_WATER_VOLUME'].data[:,tuple(ii for ii in range(5)),:]

In [ ]:
def norm_func(arr):
    return (arr - arr.min()) / (arr.max() - arr.min())  # 0 to 1

In [ ]:
ds

In [ ]:
ds['targets_WATER_VOLUME'] = ds['targets_WATER_VOLUME'].groupby("global_sites").map(norm_func)

In [ ]:
ds[f"shift_targets_WATER_VOLUME"] = ds['targets_WATER_VOLUME'] - ds['targets_WATER_VOLUME'].shift({"date": 1})

In [ ]:
ds['shift_targets_WATER_VOLUME'].sel({'global_sites':'krishnaraja_sagar'}).isel({'steps':0}).plot()#,'variable':'shift_targets_WATER_VOLUME'})

In [ ]:
(ds['shift_targets_WATER_VOLUME'].sel({'global_sites':'krishnaraja_sagar'}) / ds['shift_targets_WATER_VOLUME'].sel({'global_sites':'krishnaraja_sagar'}).std()).isel({'steps':0}).plot()

In [ ]:
ds['shift_targets_WATER_VOLUME'].sel({'global_sites':'krishnaraja_sagar'}).mean()

In [ ]:
ds['shift_targets_WATER_VOLUME'].sel({'global_sites':'krishnaraja_sagar'}).std()

In [ ]:
ds_arr = ds.to_array()

In [ ]:
ds_arr['variable']

In [ ]:
ds_arr.sel({'variable':ds_arr['variable']!='targets_WATER_VOLUME'}).shape

In [ ]:
ds_arr.shape

In [ ]:
ds_arr.sel({'variable':['targets_WATER_VOLUME']})

In [ ]:
ds['targets_WATER_VOLUME'].sel({'global_sites':['almatti','kabini']}).isel({'steps':0}).to_dict()

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(100,10))
ax.imshow(ds['forecast_tp'].sel({'global_sites':'kabini'}).isel({'steps':slice(0,15)}).data.T)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,10))
#std = ds['forecast_tp'].sel({'global_sites':'kabini'}).isel({'steps':slice(0,15)}).data.std()
ds['historic_tp'].sel({'global_sites':'kabini'}).isel({'steps':0}).plot(ax=ax)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,10))
std = ds['forecast_tp'].sel({'global_sites':'kabini'}).isel({'steps':slice(0,15)}).data.std()
ax.hist(ds['forecast_tp'].sel({'global_sites':'kabini'}).isel({'steps':slice(0,15)}).data.flatten()/std,bins=20)

In [ ]:
ds['forecast_tp'].sel({'global_sites':'kabini'}).median()

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,6))

for site in ['hemavathy','harangi','krishnaraja_sagar']:
    ds['targets_WATER_VOLUME'].sel({'global_sites':site}).isel({'steps':0}).plot(ax=ax, label=site)
    
ax.legend()

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(16,10))
ax2 = ax.twinx()

for site in ['hemavathy','harangi','krishnaraja_sagar','kabini']:
    #ds['historic_tp'].sel({'global_sites':site}).isel({'steps':0}).plot(ax=ax, label=site)
    ds['targets_WATER_VOLUME'].sel({'global_sites':site}).isel({'steps':0}).plot(ax=ax2, label=site)
    
ax.legend()

In [ ]:
ds['targets_WATER_VOLUME'].sel({'global_sites':'tattihalla'}).isel({'steps':0}).plot()

In [ ]:
len(ds_data.keys())

In [ ]:
dt_idx = (pd.to_datetime(ds['date'])>datetime(2021,1,1))&(pd.to_datetime(ds['date'])<datetime(2021,1,31))

In [ ]:
for site in ds['global_sites']:
    print (ds['targets_WATER_VOLUME'].isel({'steps':0}).sel({'global_sites':site,'date':dt_idx}))#.plot()

In [ ]:
ds['targets_WATER_VOLUME'].isel({'steps':0}).sel({'global_sites':'hirakud','date':dt_idx})#.plot()

In [ ]:
ds_data[0].keys(), ds_meta[0]

In [ ]:
for kk, data in ds_data.items():
    for kk2, vv in data.items():
        if np.isnan(vv).any():
            print (kk,kk2,vv.shape, [(ii,select_sites[ii]) for ii in np.where(np.isnan(vv).any(axis=0))[0]], ds_meta[kk])

In [ ]:
preds = xr.open_dataset('/home/lucas/h2ox-ai/experiments/sacred/104/preds.nc').compute()

In [ ]:
preds

In [ ]:
preds.isel({'step':0,'site':0})['sim-mean'].plot()

In [ ]:
preds.isel({'step':0,'site':0})['obs'].plot()

In [ ]:
var_norms = pickle.load(open('./../data/var_norms.pkl','rb'))

In [ ]:
var_norms['zscore']['shift_targets_WATER_VOLUME']

In [ ]:
5.18069319e-05*0.01480313

In [ ]:
var_norms['zscore']['shift_targets_WATER_VOLUME']['std']

In [ ]:
preds['obs']

In [ ]:
preds['obs']*var_norms['zscore']['shift_targets_WATER_VOLUME']['std'].rename({'global_sites':'site'})+var_norms['zscore']['shift_targets_WATER_VOLUME']['mean'].rename({'global_sites':'site'})

In [ ]:
norms = xr.open_dataset('/home/lucas/h2ox-ai/data/sacred/54/preds.nc').compute()

In [ ]:
ds = xr.open_dataset('/home/lucas/h2ox-ai/data/test_data_shift.nc')#.compute()

In [ ]:
def norm_func(arr):
    return (arr - arr.min()) / (arr.max() - arr.min()) # 0 to 1
ds['targets_WATER_VOLUME'] = ds['targets_WATER_VOLUME'].groupby('global_sites').map(norm_func)

In [ ]:
ds = ds.rename({'global_sites':'site','steps':'step'})

In [ ]:
preds = xr.open_dataset('/home/lucas/h2ox-ai/experiments/sacred/49/preds.nc').compute()

In [ ]:
preds['sim-frozen'].cumsum(dim='step').isel({'date':30,'site':2})

In [ ]:
preds['obs'].cumsum(dim='step').isel({'date':30,'site':2})

In [ ]:
test_chunks = [["2011-01-01","2011-12-31"],["2020-01-01","2020-12-31"]]

In [ ]:
ci_dims = ['ci-95+','ci-95-']

In [ ]:
cmap = cm.get_cmap("winter_r")
n_cols = 3
n_rows = ceil(len(preds["site"]) / n_cols) * len(test_chunks)
fig, axs = plt.subplots(n_rows, n_cols, figsize=(6 * n_cols, 3 * n_rows))
axs = axs.flatten()
_ii = 0
for chunk in test_chunks:
    date_idx = (preds["date"] >= pd.to_datetime(chunk[0])) & (
        preds["date"] <= pd.to_datetime(chunk[1])
    )
    for site in preds["site"].data:

        for step in [5, 15, 25, 50, 75, 89]:
            # print (site, step)
            hexcolor = "#" + "".join(
                f"{int(el*255):02x}" for el in cmap(int(step / 90 * 255))[:3]
            )
            # plot main
            preds.sel({"site": site, "step": step, "date": date_idx})['sim-mean'].plot(ax=axs[_ii], c=hexcolor)

            # plot CI if available
            if ci_dims is not None:
                for dim in ci_dims:
                    preds.sel({"site": site, "step": step, "date": date_idx})[dim].plot(ax=axs[_ii], c=hexcolor, ls=':')

        preds.sel({"site": site, "step": 0, "date": date_idx})["obs"].plot(
            ax=axs[_ii], c="#f200ff"
        )
        _ii += 1

In [ ]:
cmap = cm.get_cmap('winter_r')

In [ ]:
#for var in ['obs','sim']:
#    preds[var] = (
#        preds[var].cumsum(dim='step') + 
#        ds['targets_WATER_VOLUME'].sel({'date':ds['date'].isin(preds['date'])}).isel({'step':0})
#    )

In [ ]:
a = ['a','d']

In [ ]:
if a:
    print(a)

In [ ]:
preds

In [ ]:
n_cols = 3
n_rows = ceil(len(ds['site'])/n_cols)*len(test_chunks)
fig, axs = plt.subplots(n_rows, n_cols,figsize=(6*n_cols,3*n_rows))
axs = axs.flatten()
_ii = 0
for chunk in test_chunks:
    date_idx = (preds['date']>=pd.to_datetime(chunk[0]))&(preds['date']<=pd.to_datetime(chunk[1]))
    for site in preds['site'].data:
        
        for step in [0,5,10,15,25,50,75,89]:
            #print (site, step)
            rgba = cmap(int(step/90*255))
            preds.sel({'site':site,'step':step,'date':date_idx}).shift(date=step)['sim'].plot(ax=axs[_ii], c='#'+''.join(f'{int(el*255):02x}' for el in rgba[:3]))
            
        
        preds.sel({'site':site,'step':0,'date':date_idx})['obs'].plot(ax=axs[_ii], c='#f200ff')
        _ii+=1

In [ ]:
preds['obs']

In [ ]:
preds['obs'].isel({'site':1,'date':3})

In [ ]:
ds['targets_WATER_VOLUME'].isel({'steps':0,'site':0}).sel({'date':ds['date'].isin(preds['date'])})

In [ ]:
ds['targets_WATER_VOLUME'].sel({'date':ds['date'].isin(preds['date'])}).isel({'site':0,'date':2})

In [ ]:
preds['obs'].cumsum(dim='step').isel({'date':2,'site':0})

In [ ]:
(preds['obs'].cumsum(dim='step') + ds['targets_WATER_VOLUME'].sel({'date':ds['date'].isin(preds['date'])}).isel({'steps':0})).isel({'site':0,'date':2})

In [ ]:
xr.apply_ufunc(np.cumsum, preds['obs'], input_core_dims=[["date","site"]])

In [ ]:
ds.sel({'site':'bhadra','step':0})

In [ ]:
len(ds['site'])

In [ ]:
from math import ceil

In [ ]:
ds

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(6,6))
#ds.sel({'site':'bhadra','step':0,'date':dt_chunk})['obs'].plot(ax=ax, c='c')
ds.sel({'site':'bhadra','step':0,'date':dt_chunk})['obs'].plot(ax=ax, c='b')
#ds.sel({'site':'bhadra','step':5,'date':dt_chunk})['obs'].plot(ax=ax, c='c')
ds.sel({'site':'bhadra','step':0,'date':dt_chunk})['sim'].shift({'date':0}).plot(ax=ax, c='c')
ds.sel({'site':'bhadra','step':15,'date':dt_chunk})['sim'].shift({'date':15}).plot(ax=ax, c='g')
#ax.plot(
#    pd.date_range('2020-03-01',periods=90,freq='D'),
#    ds.sel({'site':'bhadra','date':pd.to_datetime('2020-03-01')})['sim'].data,
#    c='c'
#)
ax.plot(
    pd.date_range('2020-05-01',periods=90,freq='D'),
    ds.sel({'site':'bhadra','date':pd.to_datetime('2020-05-01')})['sim'].data,
    c='c'
)

In [ ]:
f'#{0:02x}{75:02x}{255:02x}'

In [ ]:
dt_chunk = (ds['date']>=pd.to_datetime(chunk[0]))&(ds['date']<=pd.to_datetime(chunk[1]))

In [ ]:
for chunk in test_chunks:
    print (ds.sel({'site':site,'step':step,'date':(ds['date']>=pd.to_datetime(chunk[0]))&(ds['date']<=pd.to_datetime(chunk[1]))}))

In [ ]:
from matplotlib import cm

In [ ]:
cmap(255)

In [ ]:
import numpy as np

In [ ]:
a = np.array([False,True,False,True,False])
b = np.array([False,True,True,False,False])
c = np.array([False,False,True,True,False])

In [ ]:
np.sum(np.stack([a,b,c]),axis=0)>0

In [ ]:
def get_exponential_weights(horizon: int):
    # exponential weighting
    wt = np.exp(np.linspace(0, 5, horizon))[::-1]
    wt = wt / np.linalg.norm(wt)
    return wt

In [ ]:
import torch

In [ ]:
def get_exponential_weights(horizon: int, clip:float) -> torch.Tensor:
    # exponential weighting
    wt = np.exp(np.linspace(0, 5, horizon))[::-1]
    wt[int(horizon*clip):] = 0
    wt = wt / np.linalg.norm(wt)
    wt = torch.from_numpy(wt)
    return wt

In [ ]:
wt = get_exponential_weights(90,0.01)

In [ ]:
wt

# torch.stack([wt]*6,dim=1).shape

In [ ]:
inp = torch.rand((90,6))
oup = torch.rand((90,6))

In [ ]:
wt * (inp - oup)**2

In [ ]:
get_exponential_weights(70).shape

In [ ]:
wt = np.exp(np.linspace(0, 5, 70))[::-1]

In [ ]:
clip=0.5

In [ ]:
wt[int(70*clip):] = 0

In [ ]:
wt / np.linalg.norm(wt)

In [ ]:
from datetime import timedelta

In [ ]:
np.repeat([np.arange(5)],10, axis=0)

In [ ]:
idx = pd.date_range('2010-01-01', '2012-01-01', freq="d")
idx.name = "date"

cols = pd.TimedeltaIndex([timedelta(days=ii) for ii in [0,5,10,15]], name="steps")

data = np.repeat([np.arange(len([0,5,10,15]))],len(idx),axis=0)/len([0,5,10,15])

df = pd.DataFrame(data=data, index=idx, columns=cols)

In [ ]:
df